In [ ]:
#Set these parameters to desired values before running the code

region_file = "observed_target_info.reg" #The region file with supernova coordinates and redshifts

fits_directory='./' #The directory to look for .fits files in   

output_file="output " #The output file name without any extension. Include a space at the end of the string.

uv_type = 'NUV' #Specify FUV or NUV observations


In [ ]:
#General use
import numpy as np
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

#For creating a list of .fits files to perform photometry on
import os

#For reading and creating tables
from astropy.table import Table
import astropy.io.ascii as asc

#For performing photometry
from astropy import units as u
from photutils import aperture_photometry, SkyCircularAperture, CircularAperture
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

#For generating a plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline


In [ ]:
#We define functions used to calculate the error in different values

'''
    Find error in the conversion factor from arcmin^2 to Kpc^2 using a black box error method.

    Args:
        val (float): A redshift value
        err (float): Error in the redshift
        
    Returns:
        error (float): Error in the conversion factor
'''

def Conv_error(val,err): 
    
    error = abs(((1/cosmo.kpc_comoving_per_arcmin(val+err))**2-(1/cosmo.kpc_comoving_per_arcmin(val-err))**2)/2).value
    return(error)


'''
    Find error in the comoving distance using a black box error method

    Args:
        val (float): A redshift value
        err (float): Error in the redshift
        
    Returns:
        error (float): Error in comoving distance 
'''

def Comov_error(val,err):
    
    error = abs(((cosmo.comoving_distance(val+err).cgs)-(cosmo.comoving_distance(val-err).cgs))/2).value           
    return(error)


'''
    Find error in the calculated flux due to error in the measured photon counts per second
    
    Args:
        val (float): Average photon counts per second
        err (float): Error in val
        
    Returns:
        error (float): Error in the flux
'''

def Flux_error(val,err):

    error = ((val*flux_conv_err)**2+(flux_conv*err)**2)**(1/2)
    return(error)


'''
    Find error in the calculated Luminosity due error in the flux and comoving distance
    
    Args:
        flux     (float): Average photon counts per second
        flux_err (float): Error in val
        dist     (float): Comoving distance
        dist_err (float): Error in comoving distance
        
    Returns:
        error (float): Error in the luminosity
'''

def Luminosity_error(flux,flux_err,dist,dist_err): #The input 'dist' is Comoving Distance

    error = ((4*np.pi*(dist**2)*flux_err)**2+(8*np.pi*dist*flux*dist_err)**2)**(1/2)
    return(error)


'''
    Find the error in surface brightness 
    
    Args:
        lum      (float): Luminosity of a supernova environment
        lum_err  (float): Error in the luminosity
        conv     (float): Conversion factor from arcmin^2 to Kpc^2 
        conv_err (float): Error in the conversion factor
        
    Returns:
        error (float): Error in the Surface brightness
'''

def Sbrightness_error(lum,lum_err,conv,conv_err): #Uncertanty in surface brightness
    
    error = ((conv*lum_err/pixel_area)**2+(lum*conv_err/pixel_area)**2+
         (lum*conv*pixel_area_err/(pixel_area**2))**2)**(1/2)
    
    return(error)


In [ ]:
'''
    Perform photometry on a .fits file using a 1kpc radius aperture. 
    
    Args:
        fits_file (str): The file path of a .fits file

    Returns:
        results (list): List of [supernova name (string), photometry value (float), exposure time (float)]
        'none'  (str) : Returned if a supernova was found, but its photometry was zero and there was no check file
        False   (bool): Returned if no supernova is found in the .fits file
'''

def photometry(fits_file):

    #open the file and create an hdulist
    hdulist = fits.open(fits_file)
    sci = hdulist[0].data
    wcs = WCS(fits_file)

    for sn in cord:

        #Define the SN location in pixels
        w = wcs.all_world2pix(cord[sn].ra,cord[sn].dec, 1)
        
        #Make sure the sn is located in the image
        if 0<w[0]<3600 and 0<w[1]<3600:

            #get exposure time and create an array of the error in each pixel
            exp_time = hdulist[0].header['EXPTIME']
            error = ((hdulist[0].data*exp_time)**(1/2))/exp_time
            
            #Find arcmin of a 1kpc radius region
            r = 2*u.kpc/cosmo.kpc_comoving_per_arcmin(float(red[sn]))
            
            #Create an aperture
            aperture = SkyCircularAperture(cord[sn], r) 
            
            #Perform photometry
            phot_table = aperture_photometry(hdulist[0], aperture, error=error)
            
            results = [sn, phot_table, exp_time]
            
            if phot_table[0][0]!=0:
                return(results)
            
            #If the photometry process returns 0, we run the function zero_check 
            #which uses a check file to determine if the aperture is in the data region
            elif zero_check(fits_file, cord[sn], r)==True:
                return(results)
            
            elif zero_check(fits_file, cord[sn], r)=='none':
                return('none')
            
    #close the hdulist
    hdulist.close()
    
    return(False)


In [ ]:
'''
    Perform photometry on a wt type .fits file and determine if there are data pixels in an aperture
    
    Args:
        fits_file (str)                                         : The file path of a .fits file
        cord      (astropy.coordinates.sky_coordinate.SkyCoord) : The (ra, dec) of a supernova in degrees
        r         (astropy.units.quantity.Quantity)             : The radius of a photometry aperture in arcmin.

    Returns:
        in_file (bool): List of [supernova name(string), photometry value (float), exposure time (float)]
        'none'  (str) : Returned if there is no check file
'''

def zero_check(fits_file, cord, r):


    if(os.path.isfile(fits_file.replace(file_key,check_file_key))):
          
        hdulist = fits.open(fits_file.replace(file_key,check_file_key))
        aperture = SkyCircularAperture(cord, r)
        phot_table = aperture_photometry(hdulist[0], aperture)
        in_file = phot_table[0][0]!=0
        return(in_file)
    
    else:
        return('none')


In [ ]:
#Set parameters that are specific to NUV or FUV observations

if 'N' in uv_type.upper():
    
    file_key = "nd-int" #A string located in all the files you want to analyze in fits_directory
    check_file_key = "nd-wt" #A string to replace file_key when running photometry on the check file
    flux_conv = 2.06*(10**(-16)) #A conversion factor from counts per second to flux
    plot_name = 'Supernova Region Brightness in the NUV' #The title of the output plot
        
elif 'F' in uv_type.upper():
    file_key = "fd-int" 
    check_file_key = "fd-wt" 
    flux_conv = 1.40*10**(-15) 
    plot_name = 'Supernova Region Brightness in the FUV'


flux_conv_err = 0 #Error in flux_conv
pixel_area=(1.5/60)**2 #Arcmin^2 per pixel - given in arcseconds, converted to arcmin, and squared to find area in arcmin^2.
pixel_area_err = 0 #Error in pixel_area


In [ ]:
#Create a list of .fits files to perform photometry on

file_list = []

for path, subdirs, files in os.walk(fits_directory):
    for name in files:
        if file_key in name: file_list.append(os.path.join(path, name))


In [ ]:
#Create dictionaries for the coordinates (in degrees) and redshift of supernova by using values from the .reg file

reg = asc.read(region_file, data_start=2, delimiter = "#", header_start=2)
cord, red = {}, {}

for row in reg: 
    cord[row[1].split(",")[0].strip('text={}')] = SkyCoord(row[0].strip('point()').replace(',',' '), unit=(u.hourangle, u.deg))
    red[row[1].split(",")[0].strip('text={}')] = row[1].split(",")[2].strip('}').replace('z=','')


In [ ]:
#Define arrays used to generate the output files

n=0
log = np.array([['Issue', 'File Path']])
out = np.array([['Sn Name', 'Red shift', 'Redshift error', 'ArcMin^2 per Kpc^2 at Redshift', 
                 'ArcMin^2 / Kpc^2 error', 'Photometry', 'Exposure Time (s)', 
                 'Photometry Error N^(1/2)/s', 'Flux (erg s-1 cm-2 A-1 px-1)', 
                 'Flux error (erg s-1 cm-2 A-1 px-1)', 'Luminosity (erg s-1 A-1 px-1)', 
                 'Luminosity Error (erg s-1 A-1 px-1)', 'Surface Brightness (erg s-1 A-1 Kpc^-2)', 
                 'Surface Brightness error (erg s-1 A-1 Kpc^-2)', 'log10 of Surface Brightness', 
                 'Error in log10', 'File Path']])

#We perform photometry on each .fits file

for fits_file in file_list:

    p = photometry(fits_file)
    
    if p==False: 
        log = np.vstack((log, np.array([['No Supernova Found', fits_file]])))
        
    elif p=='none':
        log = np.vstack((log, np.array([['No Check File', fits_file]])))
  
    else:
        n+=1
        
        #We calculate the values in the table
        
        redshift = float(red[p[0]])
        redshift_err = ((redshift/1000)**2+(300/(299792.458))**2)**(1/2) #Error in redshift taken as 1 in 1000 with an additional 300 km/s contribution from peculiar velocity
        
        arcmin = (1/cosmo.kpc_comoving_per_arcmin(redshift).value)**2 #conversion factor from arcmin^2 to Kpc^2
        arcmin_err = Conv_error(redshift, redshift_err)
        
        photom = p[1][0][0] #The photometry value
        photom_err = p[1][0][1]
        
        flux = flux_conv*photom #convert cps to flux using the conversion factor
        flux_err = Flux_error(photom,photom_err)
        
        cmd = cosmo.comoving_distance(redshift).cgs.value #Comoving Distance at Redshift (cm)
        cmd_err = Comov_error(redshift,redshift_err)
        
        lum = flux*4*np.pi*(cmd**2) #luminosity = flux*4*pi*r^2
        lum_err = Luminosity_error(flux,flux_err,cmd,cmd_err)
        
        sbrightness = (lum/pixel_area)*arcmin
        sbrightness_err = Sbrightness_error(lum,lum_err,arcmin,arcmin_err)
        
        log_sbrightness = np.log10(sbrightness)
        log_sbrightness_err = sbrightness_err/(sbrightness*np.log(10))
        
        row = np.array([[p[0] , redshift, redshift_err, arcmin, arcmin_err, photom, p[2],
                         photom_err, flux, flux_err, lum, lum_err, sbrightness, sbrightness_err,
                        log_sbrightness, log_sbrightness_err, fits_file]])
        
        out = np.vstack((out, row))
        

print('\nPhotometry finished:', n, 'supernova oberved.')

In [ ]:
#We keep only the results for each supernova with the smallest error values

out_unique = out[0,:] #We create a new array with the first row of out (the collumn headers)

for entry in out: 
    if entry[0] not in out_unique:
        out_unique = np.vstack((out_unique,entry))
    else: 
        for row in out_unique:
            if entry[0]==row[0] and entry[13]<row[13]: row=entry


In [ ]:
#We create a plot of the results and write them to a pdf file
   
t = Table(out_unique[1:], names=out_unique[0], dtype=(str, 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', 'float64', str))

ax = plt.gca()
ax.errorbar(t['Red shift'], t['Surface Brightness (erg s-1 A-1 Kpc^-2)'], xerr=None, yerr=t['Surface Brightness error (erg s-1 A-1 Kpc^-2)'], color='black', linestyle='')
ax.plot(t['Red shift'], t['Surface Brightness (erg s-1 A-1 Kpc^-2)'],'.')
ax.set_yscale('log')

plt.xlabel('Redshift')
plt.ylabel('log 10 of Surface Brightness')
plt.title(plot_name)

plt.savefig(output_file + uv_type + ' plot.pdf')
plt.show()
plt.close()



In [ ]:
#Write data to an output file

asc.write(out_unique[1:], output_file + uv_type + '.csv', names=out_unique[0], delimiter=",")
asc.write(log[1:], output_file + uv_type + ' log.csv', names=log[0], delimiter=",")
print('Script finished')
